In [1]:
projects = !gcloud config get-value project
!gcloud config set project {projects[0]}
!gcloud container clusters get-credentials two-node-cluster --zone us-central1-a

Updated property [core/project].
Fetching cluster endpoint and auth data.
kubeconfig entry generated for two-node-cluster.


In [2]:
# Get the nodes names
nodes = !kubectl get nodes
first_node = nodes[1].split()[0]
second_node = nodes[2].split()[0]
print(f"First node: {first_node}")
print(f"Second node: {second_node}")

First node: gke-two-node-cluster-node-pool-1-93990693-mr7z
Second node: gke-two-node-cluster-node-pool-1-93990693-vkd4


In [3]:
# Change node names in yaml files

import yaml

dir_yaml = 'deploy_real_kubernetes/'

# FIRST NODE
with open(dir_yaml + 'deploy_micro_inf.yaml', 'r') as filename:
    micro_inf_configs = list(yaml.safe_load_all(filename))
micro_inf_configs[0]['spec']['template']['spec']['nodeName'] = first_node
with open(dir_yaml + 'deploy_micro_inf.yaml', 'w') as filename:
    yaml.safe_dump_all(micro_inf_configs, filename)

with open(dir_yaml + '04-mlflow.yaml', 'r') as filename:
    mlflow_configs = list(yaml.safe_load_all(filename))
mlflow_configs[2]['spec']['template']['spec']['nodeName'] = first_node
with open(dir_yaml + '04-mlflow.yaml', 'w') as filename:
    yaml.safe_dump_all(mlflow_configs, filename)

# SECOND NODE
with open(dir_yaml + '01-zookeeper.yaml', 'r') as filename:
    zookeeper_configs = list(yaml.safe_load_all(filename))
zookeeper_configs[1]['spec']['template']['spec']['nodeName'] = second_node
with open(dir_yaml + '01-zookeeper.yaml', 'w') as filename:
    yaml.safe_dump_all(zookeeper_configs, filename)

with open(dir_yaml + '02-kafka.yaml', 'r') as filename:
    kafka_configs = list(yaml.safe_load_all(filename))
kafka_configs[1]['spec']['template']['spec']['nodeName'] = second_node
with open(dir_yaml + '02-kafka.yaml', 'w') as filename: 
    yaml.safe_dump_all(kafka_configs, filename)

with open(dir_yaml + 'deploy_micro_up.yaml', 'r') as filename:
    micro_up_configs = list(yaml.safe_load_all(filename))
micro_up_configs[0]['spec']['template']['spec']['nodeName'] = second_node
with open(dir_yaml + 'deploy_micro_up.yaml', 'w') as filename:
    yaml.safe_dump_all(micro_up_configs, filename)

In [4]:
# Apply yaml files
dir_name = 'deploy_real_kubernetes'
import os

files = [f for f in os.listdir(dir_name) if os.path.isfile(os.path.join(dir_name, f))]
for file in files:
    if file.endswith('.yaml'):
        print(f'Applying {file}...')
        !kubectl apply -f {os.path.join(dir_name, file)}
    else:
        print(f'Skipping {file}, not a YAML file.')

Applying 00-namespace.yaml...
namespace/kafka created
Applying 01-zookeeper.yaml...
service/zookeeper-service created
deployment.apps/zookeeper created
Applying 02-kafka.yaml...
service/kafka-service created
deployment.apps/kafka-broker created
Applying 04-mlflow.yaml...
persistentvolumeclaim/mlflow-pvc created
service/mlflow created
deployment.apps/mlflow created
Applying api_inference_svc.yaml...


Error from server: error when creating "deploy_real_kubernetes\\04-mlflow.yaml": admission webhook "warden-mutating.common-webhooks.networking.gke.io" denied the request: GKE Warden rejected the request because it violates one or more mutators.
Violations details: ["error querying GCE PD volume mlflow-disk: disk is not found"]
Requested by user: 'AdilMuhammad706@gmail.com', groups: 'system:authenticated'.


service/api-inferencia created
Applying api_update_svc.yaml...
service/api-update created
Applying deploy_micro_inf.yaml...
deployment.apps/api-inferencia created
persistentvolumeclaim/api-inferencia-pvc created
Applying deploy_micro_up.yaml...


Error from server: error when creating "deploy_real_kubernetes\\deploy_micro_inf.yaml": admission webhook "warden-mutating.common-webhooks.networking.gke.io" denied the request: GKE Warden rejected the request because it violates one or more mutators.
Violations details: ["error querying GCE PD volume api-update-disk: disk is not found"]
Requested by user: 'AdilMuhammad706@gmail.com', groups: 'system:authenticated'.


deployment.apps/api-update created
persistentvolumeclaim/api-update-pvc created
Skipping node1.txt, not a YAML file.


Error from server: error when creating "deploy_real_kubernetes\\deploy_micro_up.yaml": admission webhook "warden-mutating.common-webhooks.networking.gke.io" denied the request: GKE Warden rejected the request because it violates one or more mutators.
Violations details: ["error querying GCE PD volume api-update-disk: disk is not found"]
Requested by user: 'AdilMuhammad706@gmail.com', groups: 'system:authenticated'.


In [55]:
# Copy Files to Pods

# pods = !kubectl get pods | findstr /R "upd"

# Check if pods are running
# !kubectl exec {pods[0].split()[0]} -- ls /app/

pods = !kubectl get pods | findstr /R "upd"
# !kubectl cp ../temp/upd/API_update_V8.1.py {pods[0].split()[0]}:/app/API_update_V8.1.py 
!kubectl cp ../temp/upd/API_update_V8.1_HT.py {pods[0].split()[0]}:/app/API_update_V8.1_HT.py 
# !kubectl cp ../temp/upd/API_update_V8.1_debug.py {pods[0].split()[0]}:/app/API_update_V8.1_debug.py 
# !kubectl cp ../../Datasets/real_usage/AGR_a_first_train.csv {pods[0].split()[0]}:/app/AGR_a_first_train.csv
# !kubectl cp ../../Datasets/real_usage/AGR_g_first_train.csv {pods[0].split()[0]}:/app/AGR_g_first_train.csv

# pods = !kubectl get pods | findstr /R "inf"
# !kubectl cp ../temp/inf/API_inferencia_V8.0.py {pods[0].split()[0]}:/app/API_inferencia_V8.0.py


In [57]:
# Run the scripts

import os
import threading

# update_pods = !kubectl get pods | findstr /R "upd"

# def run_update_script(pod_name):
#     os.system(f"kubectl exec {pod_name} -- nohup python3 /app/API_update_V8.1_HT.py > NUL 2>&1 &")
#     # os.system(f"kubectl exec {pod_name} -- nohup python3 /app/API_update_V8.1.py > NUL 2>&1 &")
# t = threading.Thread(target=run_update_script, args=(update_pods[0].split()[0],))
# t.start()

def run_inference_script(pod_name, script_path):
    os.system(f"kubectl exec {pod_name} -- nohup python3 {script_path} > NUL 2>&1 &")

inference_pods = !kubectl get pods | findstr /R "inf"
for pod in inference_pods:
    pod_name = pod.split()[0]
    script_path = "/app/API_inferencia_V8.0.py"
    t = threading.Thread(target=run_inference_script, args=(pod_name, script_path))
    t.start()

In [10]:
# RESTART KAFKA
!kubectl delete -f deploy_real_kubernetes/02-kafka.yaml
!kubectl apply -f deploy_real_kubernetes/02-kafka.yaml

service "kafka-service" deleted
deployment.apps "kafka-broker" deleted
service/kafka-service created
deployment.apps/kafka-broker created


In [27]:
# Health check for Kafka pod

# Get Kafka pods
kafka_pods = !kubectl get pods -n kafka | findstr /R "kafka"

if kafka_pods:
    for pod in kafka_pods:
        pod_name = pod.split()[0]
        print(f"Checking Kafka pod: {pod_name}")
        command = "kubectl get pod {} -n kafka -o jsonpath='{{.status.phase}}'".format(pod_name)
        status = !{command}
        print(f"Status: {status[0]}")
        if status[0].strip("'") == "Running":
            print(f"Kafka pod {pod_name} is running.")
        else:
            print(f"Kafka pod {pod_name} is NOT running.")
else:
    print("No Kafka pods found.")

Checking Kafka pod: kafka-broker-7d45898578-q5nm5
Status: 'Running'
Kafka pod kafka-broker-7d45898578-q5nm5 is running.


In [58]:
# [SKIP] Kill the scripts
inference_pods = !kubectl get pods | findstr /R "inf"
update_pods = !kubectl get pods | findstr /R "upd"
for pod in inference_pods:
    !kubectl exec {pod.split()[0]} -- pkill -f API
for pod in update_pods:
    !kubectl exec {pod.split()[0]} -- pkill -f API

In [8]:
# See python processes running in the pod
update_pods = !kubectl get pods | findstr /R "upd"
inference_pods = !kubectl get pods | findstr /R "inf"
print("Checking Python processes in pods:")
# for pod in inference_pods:
#     print(f"Pod: {pod.split()[0]}")
#     !kubectl exec {pod.split()[0]} -- ps aux | findstr /R "python"
print(f"Pod: {inference_pods[0].split()[0]}")
!kubectl exec {inference_pods[0].split()[0]} -- ps aux | findstr /R "python"


for pod in update_pods:
    print(f"Pod: {pod.split()[0]}")
    !kubectl exec {pod.split()[0]} -- ps aux | findstr /R "python"

Checking Python processes in pods:
Pod: api-inferencia-6688fcd87d-242pw
root          20 10.1  0.5 5064128 187940 ?      Ssl  04:19   3:00 python3 /app/API_inferencia_V8.0.py
Pod: api-update-59d77fff5f-zf9zt
root          65 11.8  0.6 1211968 230060 ?      Ssl  04:19   3:32 python3 /app/API_update_V8.1.py
root        2604  0.0  0.0      0     0 ?        Z    04:33   0:00 [python3] <defunct>
root        2622  0.0  0.0      0     0 ?        Z    04:33   0:00 [python3] <defunct>
root        2640  0.0  0.0      0     0 ?        Z    04:33   0:00 [python3] <defunct>


In [24]:
# Check if program is working
update_pods = !kubectl get pods | findstr /R "upd"
!kubectl exec {update_pods[0].split()[0]} -- ls
# !kubectl exec {update_pods[0].split()[0]} -- cat api_update.log

AGR_a_first_train.csv
AGR_g_first_train.csv
API_update_V8.1.py
api_update.log
lost+found


In [3]:
# RUN LOAD TESTING INSTANCE VM
!gcloud compute instances start load-testing-instance --zone=us-central1-a

Starting instance(s) load-testing-instance...
...........................................................................................done.
Updated [https://compute.googleapis.com/compute/v1/projects/fast-learner-project/zones/us-central1-a/instances/load-testing-instance].
Instance internal IP is 10.128.0.17
Instance external IP is 104.197.3.201


In [6]:
# RUN LOAD TESTING CONTAINER
# docker_command = "docker start -d --name load-testing-container auliadil/load-testing-rodrigues:v1 tail -f /dev/null"
docker_command = "docker start load-testing-container"
gcloud_template = "gcloud compute ssh load-testing-instance --zone=us-central1-a"
!{gcloud_template} --command="{docker_command}"

load-testing-container


In [7]:
# CHECK IF CONTAINER IS RUNNING
zone = "us-central1-a"
!gcloud compute ssh load-testing-instance --zone={zone} --command="docker ps"

CONTAINER ID   IMAGE                                COMMAND               CREATED       STATUS          PORTS     NAMES
032ca7df2590   auliadil/load-testing-rodrigues:v1   "tail -f /dev/null"   10 days ago   Up 22 seconds             load-testing-container


In [ ]:
!gcloud compute instances stop load-testing-instance --zone=us-central1-a

Stopping instance(s) load-testing-instance...
.....................................................................................................................................................................................................................................................................................................................................................................done.
Updated [https://compute.googleapis.com/compute/v1/projects/fast-learner-project/zones/us-central1-a/instances/load-testing-instance].


: 